In [1]:
import os
import re
from dotenv import load_dotenv
load_dotenv()
from typing import TypeVar, NamedTuple
from pydantic import BaseModel
from sqltypes import PydanticModel
from kv.sql import SQLKV
T = TypeVar('T')
from sqlalchemy import create_engine
from sqlalchemy.orm import DeclarativeBase, mapped_column, Mapped

In [2]:
class MyUser(BaseModel):
  name: str
  age: int

engine = create_engine('sqlite:///test.db')

In [9]:
kv = SQLKV(MyUser, engine)
# await kv.insert('hello', MyUser(name='hey', age=42))
await kv.read('hello')

Right(value=MyUser(name='hey', age=42), tag='right')

In [2]:
conn_str = os.environ['BLOB_CONN_STR']